In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../data"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv']


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

In [3]:
agg_cols = ['groupId', 'killPlace', 'killStreaks', 'walkDistance', 'winPlacePerc']
agg_func = {'groupId':'count', 'killPlace':['min','max'], 'killStreaks':'max', 'walkDistance':'sum' , 'winPlacePerc': 'max'}

match_0 = df_train.query('matchId == 2').reset_index()
print(match_0.keys())
print(match_0['numGroups'][0])
#print(match_0.query('groupId == 878228'))# 21')
match_0[agg_cols]\
.groupby(['groupId'])\
.agg(agg_func)\
.sort_values(by=[('winPlacePerc', 'max')], ascending=False)

Index(['index', 'Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'maxPlace', 'numGroups', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints',
       'winPlacePerc'],
      dtype='object')
28


groupId killPlace     killStreaks walkDistance winPlacePerc
          count       min max         max          sum          max
groupId                                                            
878216        4         1  47           2   12648.0000       1.0000
878227        3         4  20           2   11103.0000       0.9630
878237        4        11  48           2   11113.0000       0.9259
878238        3         6  24           1    3762.0000       0.8889
878239        4        12  49           2    6772.2000       0.8519
878240        2        27  50           1    5453.0000       0.8148
878241        3        51  53           0   10263.0000       0.7778
878242        3        13  55           1    8690.0000       0.7407
878243        5         7  56           2   10254.3000       0.7037
878217        2         5  15           1    2938.0000       0.6667
878218        1        57  57           0    1745.0000       0.6296
878219        3        58  60           0    5502.0000       0.5926
878220        5        61  65           0    7315.1000       0.5556
878221        3         2  31           2     800.8900       0.5185
878222        3        16  67           1     828.5500       0.4815
878223        4        17  69           1    1196.8000       0.4444
878224        1        33  33           1     197.2000       0.4074
878225        5         8  70           2     725.7200       0.3704
878226        3         9  71           1     633.4000       0.3333
878228        5        38  73           1     673.2800       0.2963
878229        5        41  76           1     868.9224       0.2593
878230        2        43  77           1     287.2000       0.2222
878231        1        18  18           1      56.0300       0.1852
878232        2        44  78           1     143.4820       0.1481
878233        2        79  80           0     129.1500       0.1111
878234        5        45  84           1     172.4200       0.0741
878235        7        46  90           1     228.0510       0.0370
878236        3        91  93           0      46.3560       0.0000

In [4]:
"""
it is a team game, scores within the same group is same, so let's get the feature of each group
"""
df_train_size = df_train.groupby(['matchId','groupId']).size().reset_index(name='group_size')
df_test_size = df_test.groupby(['matchId','groupId']).size().reset_index(name='group_size')

df_train_mean = df_train.groupby(['matchId','groupId']).mean().reset_index()
df_test_mean = df_test.groupby(['matchId','groupId']).mean().reset_index()

df_train_max = df_train.groupby(['matchId','groupId']).max().reset_index()
df_test_max = df_test.groupby(['matchId','groupId']).max().reset_index()

df_train_min = df_train.groupby(['matchId','groupId']).min().reset_index()
df_test_min = df_test.groupby(['matchId','groupId']).min().reset_index()

In [5]:
df_train_size.query('matchId == 0')

,matchId,groupId,group_size
0,0,1,3
1,0,2,3
2,0,3,1
3,0,4,1
4,0,5,3
5,0,6,3
6,0,7,1
7,0,8,4
8,0,9,4
9,0,10,7


In [5]:
"""
although you are a good game player, 
but if other players of other groups in the same match is better than you, you will still get little score
so let's add the feature of each match
"""
df_train_match_mean = df_train.groupby(['matchId']).mean().reset_index()
df_test_match_mean = df_test.groupby(['matchId']).mean().reset_index()

df_train = pd.merge(df_train, df_train_mean, suffixes=["", "_mean"], how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_mean, suffixes=["", "_mean"], how='left', on=['matchId', 'groupId'])
del df_train_mean
del df_test_mean

df_train = pd.merge(df_train, df_train_max, suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_max, suffixes=["", "_max"], how='left', on=['matchId', 'groupId'])
del df_train_max
del df_test_max

df_train = pd.merge(df_train, df_train_min, suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_min, suffixes=["", "_min"], how='left', on=['matchId', 'groupId'])
del df_train_min
del df_test_min

df_train = pd.merge(df_train, df_train_match_mean, suffixes=["", "_match_mean"], how='left', on=['matchId'])
df_test = pd.merge(df_test, df_test_match_mean, suffixes=["", "_match_mean"], how='left', on=['matchId'])
del df_train_match_mean
del df_test_match_mean

df_train = pd.merge(df_train, df_train_size, how='left', on=['matchId', 'groupId'])
df_test = pd.merge(df_test, df_test_size, how='left', on=['matchId', 'groupId'])
del df_train_size
del df_test_size

target = 'winPlacePerc'
train_columns = list(df_test.columns)

""" remove some columns """
train_columns.remove("Id")
train_columns.remove("matchId")
train_columns.remove("groupId")
train_columns.remove("Id_mean")
train_columns.remove("Id_max")
train_columns.remove("Id_min")
train_columns.remove("Id_match_mean")

In [ ]:
"""
in this game, team skill level is more important than personal skill level 
maybe you are a good player, but if your teammates is bad, you will still lose
so let's remove the features of each player, just select the features of group and match
"""
train_columns_new = []
for name in train_columns:
    if '_' in name:
        train_columns_new.append(name)
train_columns = train_columns_new    
print(train_columns)

# train_columns = ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 
#                 'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks', 
#                 'longestKill', 'maxPlace', 'numGroups', 'revives','rideDistance', 
#                 'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 
#                 'weaponsAcquired', 'winPoints']

x_train = df_train[train_columns]
x_test = df_test[train_columns]
y = df_train[target]

del df_train

x_train = np.array(x_train, dtype=np.float64)
x_test = np.array(x_test, dtype=np.float64)
y = np.array(y, dtype=np.float64)

from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(x_train)
# scaler = preprocessing.QuantileTransformer().fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

# x_train = x_train*2 - 1
# x_test = x_test*2 - 1
y = y*2 - 1

print("x_train", x_train.shape, x_train.min(), x_train.max())
print("x_test", x_test.shape, x_test.min(), x_test.max())
print("y", y.shape, y.min(), y.max())

x_test = np.clip(x_test, a_min=-1, a_max=1)
print("x_test", x_test.shape, x_test.min(), x_test.max())

from ultimate.mlp import MLP 
mlp = MLP(layer_size=[x_train.shape[1], 28, 28, 28, 1], regularization=1, output_shrink=0.1, output_range=[-1,1], loss_type="hardmse")

"""
train 15 epoches, batch_size=1, SGD
"""
mlp.train(x_train, y, verbose=2, iteration_log=20000, rate_init=0.08, rate_decay=0.8, epoch_train=15, epoch_decay=1)
pred = mlp.predict(x_test)
pred = pred.reshape(-1)

pred = (pred + 1) / 2
# df_test['winPlacePerc'] = np.clip(pred, a_min=0, a_max=1)

df_test['winPlacePercPred'] = np.clip(pred, a_min=0, a_max=1)
# df_test['winPlacePercPred'] = pred
aux = df_test.groupby(['matchId','groupId'])['winPlacePercPred'].agg('mean').groupby('matchId').rank(pct=True).reset_index()
aux.columns = ['matchId','groupId','winPlacePerc']
df_test = df_test.merge(aux, how='left', on=['matchId','groupId'])
    
submission = df_test[['Id', 'winPlacePerc']]

submission.to_csv('submission.csv', index=False)